<a href="https://colab.research.google.com/github/Joykareko/AWS-Udacity-Capstone-Project/blob/main/Arvato_Project_Workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [32]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


# magic word for producing visualizations in notebook
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
! ls ../../data/Term2/capstone/arvato_data/

Udacity_AZDIAS_052018.csv     Udacity_MAILOUT_052018_TEST.csv
Udacity_CUSTOMERS_052018.csv  Udacity_MAILOUT_052018_TRAIN.csv


In [ ]:
! cp ../../data/Term2/capstone/arvato_data/* . -fv

'../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv' -> './Udacity_AZDIAS_052018.csv'
'../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv' -> './Udacity_CUSTOMERS_052018.csv'
'../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv' -> './Udacity_MAILOUT_052018_TEST.csv'
'../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv' -> './Udacity_MAILOUT_052018_TRAIN.csv'


In [ ]:
! zip all.zip * .csv

	zip warning: name not matched: .csv
  adding: Arvato Project Workbook.ipynb (deflated 68%)
  adding: Arvato-Project-Workbook-zh.ipynb (deflated 58%)
  adding: DIAS Attributes - Values 2017.xlsx (deflated 10%)
  adding: DIAS Information Levels - Attributes 2017.xlsx (deflated 15%)
  adding: Udacity_AZDIAS_052018.csv (deflated 78%)
  adding: Udacity_CUSTOMERS_052018.csv (deflated 81%)
  adding: Udacity_MAILOUT_052018_TEST.csv (deflated 79%)
  adding: Udacity_MAILOUT_052018_TRAIN.csv (deflated 79%)
  adding: terms_and_conditions/ (stored 0%)


## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [40]:
# load in the data
azdias = pd.read_csv('/content/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('/content/Udacity_CUSTOMERS_052018.csv', sep=';')
attributes = pd.read_excel('/content/DIAS Attributes - Values 2017.xlsx', header=1)
print('Data Loaded Successfully.......!')

Data Loaded Successfully.......!


### Checking out the General populations' data (Azdias)

In [10]:
#checking out the data, first 3 rows
#The data has alot of missing values which will need cleanup
azdias.head(3)

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,...,REGIOTYP,RELAT_AB,RETOURTYP_BK_S,RT_KEIN_ANREIZ,RT_SCHNAEPPCHEN,RT_UEBERGROESSE,SEMIO_DOM,SEMIO_ERL,SEMIO_FAM,SEMIO_KAEM,SEMIO_KRIT,SEMIO_KULT,SEMIO_LUST,SEMIO_MAT,SEMIO_PFLICHT,SEMIO_RAT,SEMIO_REL,SEMIO_SOZ,SEMIO_TRADV,SEMIO_VERT,SHOPPER_TYP,SOHO_KZ,STRUKTURTYP,TITEL_KZ,UMFELD_ALT,UMFELD_JUNG,UNGLEICHENN_FLAG,VERDICHTUNGSRAUM,VERS_TYP,VHA,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,1.0,1.0,5.0,5.0,5.0,5.0,0,0,10,0,0,0,10,10,NaN,0,0,0,...,NaN,NaN,5.0,1.0,4.0,1.0,6.0,3.0,6.0,6.0,7.0,3.0,5.0,5.0,5.0,4.0,7.0,2.0,3.0,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2.0
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,0.0,0.0,2.0,12.0,0.0,3.0,6.0,8A,8,51,5.0,1.0,5.0,5.0,2.0,3.0,1.0,1.0,0,0,10,0,0,0,10,10,NaN,0,0,0,...,3.0,4.0,1.0,5.0,3.0,5.0,7.0,2.0,4.0,4.0,4.0,3.0,2.0,3.0,7.0,6.0,4.0,5.0,6.0,1.0,3.0,1.0,2.0,0.0,3.0,3.0,1.0,0.0,2.0,0.0,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5.0,2.0,1.0
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,0.0,0.0,1.0,7.0,0.0,3.0,2.0,4C,4,24,3.0,2.0,4.0,4.0,1.0,3.0,2.0,2.0,0,0,10,0,0,0,10,10,0.0,0,0,0,...,2.0,2.0,3.0,5.0,4.0,5.0,7.0,6.0,1.0,7.0,7.0,3.0,4.0,3.0,3.0,4.0,3.0,4.0,3.0,4.0,2.0,0.0,3.0,0.0,2.0,5.0,0.0,1.0,1.0,0.0,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5.0,2.0,3.0


In [ ]:
#checking out the number of columns and rows
azdias.shape

(891221, 366)

In [ ]:
#checking out the general info on columns
#we have 6 columns that have categorical data
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(93), object(6)
memory usage: 2.4+ GB


In [ ]:
#checking out which columns are these
list(azdias.select_dtypes(include='object').columns)

['CAMEO_DEU_2015',
 'CAMEO_DEUG_2015',
 'CAMEO_INTL_2015',
 'D19_LETZTER_KAUF_BRANCHE',
 'EINGEFUEGT_AM',
 'OST_WEST_KZ']

In [ ]:
#lets check out the number of categories
#the dataset has 6 features that are categorical. Each feature has different categories, some with unknown values
azdias.select_dtypes(include='object').nunique()

CAMEO_DEU_2015                45
CAMEO_DEUG_2015               19
CAMEO_INTL_2015               43
D19_LETZTER_KAUF_BRANCHE      35
EINGEFUEGT_AM               5162
OST_WEST_KZ                    2
dtype: int64

In [ ]:
#checking out stats features for the 360 numerical features
#the mean values are very varied. We may need to preprocess them before modelling.
#we also have alot of missing data.
azdias.describe()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
count,8.912210e+05,891221.000000,817722.000000,817722.000000,81058.000000,29499.000000,6170.000000,1205.000000,628274.000000,798073.000000,...,770025.000000,815304.000000,815304.000000,815304.000000,783619.000000,817722.000000,798073.000000,891221.000000,891221.000000,891221.000000
mean,6.372630e+05,-0.358435,4.421928,10.864126,11.745392,13.402658,14.476013,15.089627,13.700717,8.287263,...,2.417322,6.001214,7.532130,5.945972,3.933406,7.908791,4.052836,3.362438,1.522098,2.777398
std,2.572735e+05,1.198724,3.638805,7.639683,4.097660,3.243300,2.712427,2.452932,5.079849,15.628087,...,1.166572,2.856091,3.247789,2.771464,1.964701,1.923137,1.949539,1.352704,0.499512,1.068775
min,1.916530e+05,-1.000000,1.000000,0.000000,2.000000,2.000000,4.000000,7.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
25%,4.144580e+05,-1.000000,1.000000,0.000000,8.000000,11.000000,13.000000,14.000000,11.000000,1.000000,...,2.000000,3.000000,5.000000,4.000000,2.000000,8.000000,3.000000,3.000000,1.000000,2.000000
50%,6.372630e+05,-1.000000,3.000000,13.000000,12.000000,14.000000,15.000000,15.000000,14.000000,4.000000,...,2.000000,6.000000,8.000000,6.000000,4.000000,9.000000,3.000000,3.000000,2.000000,3.000000
75%,8.600680e+05,-1.000000,9.000000,17.000000,15.000000,16.000000,17.000000,17.000000,17.000000,9.000000,...,3.000000,9.000000,10.000000,8.000000,6.000000,9.000000,5.000000,4.000000,2.000000,4.000000
max,1.082873e+06,3.000000,9.000000,21.000000,18.000000,18.000000,18.000000,18.000000,25.000000,595.000000,...,4.000000,11.000000,13.000000,11.000000,6.000000,9.000000,8.000000,6.000000,2.000000,9.000000


In [5]:
#number of unknown values in general
unknowns=azdias.isnull().sum().sum()
print('Total Unknown Values for the Population Dataset is: ',unknowns)

Total Unknown Values for the Population Dataset is:  16902223


These are alot of unknowns.
Lets check the data description to check on the categorical features further for azdias.

### Checking out the Attributes data/information

In [13]:
attributes.head(2)

,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,NaN,0,no classification possible


In [14]:
attributes.tail(2)

,Unnamed: 0,Attribute,Description,Value,Meaning
2256,NaN,NaN,NaN,5,seeking orientation
2257,NaN,NaN,NaN,6,indifferent


The above data shows that there are missing values inform of (-1,0,9 and even 10)
Lets see exactly that below

In [ ]:
attributes[attributes.Meaning.str.contains('know',na=False)]

,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
5,NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
11,NaN,ALTER_HH,main age within the household,0,unknown / no main age detectable
33,NaN,ANREDE_KZ,gender,"-1, 0",unknown
40,NaN,BALLRAUM,distance to next urban centre,-1,unknown
48,NaN,BIP_FLAG,business-flag indicating companies in the buil...,-1,unknown
51,NaN,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,-1,unknown
105,NaN,CAMEO_DEUINTL_2015,CAMEO classification 2015 - international typo...,-1,unknown
131,NaN,CJT_GESAMTTYP,customer journey typology,0,unknown
138,NaN,D19_BANKEN_ANZ_12,transaction activity BANKS in the last 12 months,0,no transactions known


Thats alot of unknowns. We need to convert them to NaNs and then get a list of missing values afresh

Lets explore the customers data as well.

### Checking out the customers data characteristics

In [ ]:
#displaying 3 rows of customers dataset
customers.head(3)

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4


In [ ]:
customers.shape

(191652, 369)

In [ ]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(8)
memory usage: 539.5+ MB


We have 8 categorical type, features in the customers dataset

In [ ]:
#checking out which columns are these
list(customers.select_dtypes(include='object').columns)

['CAMEO_DEU_2015',
 'CAMEO_DEUG_2015',
 'CAMEO_INTL_2015',
 'D19_LETZTER_KAUF_BRANCHE',
 'EINGEFUEGT_AM',
 'OST_WEST_KZ',
 'PRODUCT_GROUP',
 'CUSTOMER_GROUP']

In [ ]:
#lets check out the number of categories
#the dataset has 6 features that are categorical. Each feature has different categories, some with unknown values
customers.select_dtypes(include='object').nunique()

CAMEO_DEU_2015                45
CAMEO_DEUG_2015               19
CAMEO_INTL_2015               43
D19_LETZTER_KAUF_BRANCHE      35
EINGEFUEGT_AM               3034
OST_WEST_KZ                    2
PRODUCT_GROUP                  3
CUSTOMER_GROUP                 2
dtype: int64

In [ ]:
customers.describe()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
count,191652.000000,191652.000000,145056.000000,145056.000000,11766.000000,5100.000000,1275.000000,236.000000,139810.000000,141725.000000,...,143781.000000,143781.000000,143781.000000,137910.000000,145056.000000,141725.000000,191652.000000,191652.000000,191652.000000,191652.000000
mean,95826.500000,0.344359,1.747525,11.352009,12.337243,13.672353,14.647059,15.377119,10.331579,4.965863,...,4.374417,4.564769,3.168868,4.152716,8.646371,3.723133,2.576806,0.090247,1.376432,3.060907
std,55325.311233,1.391672,1.966334,6.275026,4.006050,3.243335,2.753787,2.307653,4.134828,14.309694,...,2.924355,2.887035,2.233516,1.974375,1.154001,2.095540,1.168486,0.286536,0.484492,1.086254
min,1.000000,-1.000000,1.000000,0.000000,2.000000,2.000000,5.000000,8.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
25%,47913.750000,-1.000000,1.000000,8.000000,9.000000,11.000000,13.000000,14.000000,9.000000,1.000000,...,2.000000,2.000000,1.000000,2.000000,9.000000,2.000000,1.000000,0.000000,1.000000,3.000000
50%,95826.500000,0.000000,1.000000,11.000000,13.000000,14.000000,15.000000,16.000000,10.000000,1.000000,...,4.000000,4.000000,3.000000,5.000000,9.000000,3.000000,3.000000,0.000000,1.000000,3.000000
75%,143739.250000,2.000000,1.000000,16.000000,16.000000,16.000000,17.000000,17.000000,13.000000,4.000000,...,7.000000,7.000000,4.000000,6.000000,9.000000,5.000000,3.000000,0.000000,2.000000,4.000000
max,191652.000000,3.000000,9.000000,21.000000,18.000000,18.000000,18.000000,18.000000,25.000000,523.000000,...,11.000000,13.000000,11.000000,6.000000,9.000000,8.000000,6.000000,1.000000,2.000000,9.000000


In [4]:
#number of unknown values in general
unknowns2=customers.isnull().sum().sum()
print('Total Unknown Values for the Customers Dataset is: ',unknowns2)

Total Unknown Values for the Customers Dataset is:  13864522


In [51]:
customers.CAMEO_INTL_2015.value_counts()

14      369
24      364
41      216
43      182
15      177
25      160
51      153
13      139
22      135
14.0    129
23      118
24.0    110
45       97
34       95
54       95
55       89
41.0     86
31       79
12       78
43.0     74
25.0     69
51.0     67
15.0     61
44       57
45.0     52
22.0     47
34.0     46
13.0     46
23.0     37
35       36
32       31
12.0     29
33       27
54.0     22
44.0     19
52       18
55.0     16
32.0     14
31.0     14
35.0     12
33.0      9
52.0      8
XX        2
Name: CAMEO_INTL_2015, dtype: int64

### Cleaning the Datasets

After assessing the datasets, the below steps will be followed inorder to clean the datasets:

1. Drop all columns null/missing values with 30% missing data.
2. Replace missing values(-1/0/9/10/etc) through data imputation.
        a. Numerical features with Median (median value)
        b. Categorial features with Mode(most common)


In [41]:
#to accelerate the runtime, we will use a sample of the data.
population = azdias.sample(n=5000)
customers= customers.sample(n=5000)
print(population.shape,customers.shape)

(5000, 366) (5000, 369)


In [42]:
def drop_cols(df):
    '''
    function that drops columns that 
    have more than 30% data that is missing/contains NaNs
    '''
    #finding columns with 60% missing data
    #print("==========================")
    print('Old Data shape: ',df.shape)
    
    df_1 = (df.isnull().sum()/df.shape[0])>0.3
    df_2 = df[df.columns[~df_1]]
    print('=============================')
    print("New Data shape: ", df_2.shape)
    print('=============================')
    
    #we now have a new dataset;
    #we also need to check row wise for 60% missing data
    #df_3 = (df_2.isnull().sum(axis=0)/df_2.shape[0])>0.4
    #df_4 = df_2[df_2.columns[~df_3]]
    #print('========================================')
    #print('Final Data shape after row deletion: ', df_4.shape)
    return df_2

In [43]:
def identify_nas(attributes):
    '''
    function that shows which values are unknowns in the
    attributes dataframe
    '''
    df=attributes.copy()
    df = df[df.Meaning.str.contains('know',na=False)].reset_index(drop=True)
    
    s = df['Value'].str.split(',').apply(pd.Series, 1).stack() #splits the valaues in that column
    s.index = s.index.droplevel(1)
    s.name='New_value'
    df = df.join(s)  #joins with the new value
    df.loc[df.New_value.isna(),'New_value']=df[~df.Value.str.contains(',',na=False)].Value  #locates the values
    df.reset_index(drop=True,inplace=True)
    df['New_value']=df['New_value'].astype(float)
    
    return df

def unknown_nas(df,new_attributes):

  '''
    this function replaces all unknowns(-1,0,9,10) with NaN
    '''
  df_1 = df.copy()
  cols = df_1.columns.intersection(new_attributes.columns)
  for i in range(len(new_attributes[cols])):
    value = new_attributes.loc[i,'New_value']
    col = new_attributes.loc[0,'Attribute']
    mask = df_1[col] == value
    df_1[mask]= np.nan
        
    return df_1

In [53]:
#replacing all NaNs with median(numeical features) and Mode(categorical features)
#Median is preferable to mean due to skewness

def impute_nas(df,new_attributes):
    '''
    this function performs data imputation on the remaining dataset
    '''
    dt=new_attributes.copy()
    data=df.copy()
    
    dt=dt[dt.Meaning.str.contains('numeric',na=False)]

    print("Number of Nas before filling : {}".format(data.isnull().sum().sum()))
    # Numerical features => Median
    mask = dt.Attribute.values
    data[mask]= data[mask].fillna(data.median())
    print("Number of Nas after filling Numeric features : {}".format(data.isnull().sum().sum()))
    
    # Categorical features => Most frequent (Mode)
    mask = data.columns.difference(mask)
    data[mask]= data[mask].fillna(data.mode().iloc[0])
    print("Number of Nas after  filling All features: {}".format(data.isnull().sum().sum()))
    
    return data

In [60]:
#lets drop these columns that have too many categories
#lets also scale the numerical values

def encode_data(df,new_attributes):

    cols = ["CAMEO_DEUG_2015", "CAMEO_INTL_2015"]
    df[cols] = df[cols].replace({"X": np.nan, "XX": np.nan})
    df[cols] = df[cols].astype(float)
    df.fillna(0,inplace=True)

    # drop some features
    cols= ['CAMEO_DEU_2015','D19_LETZTER_KAUF_BRANCHE','EINGEFUEGT_AM','OST_WEST_KZ','PRODUCT_GROUP','CUSTOMER_GROUP']
    df=df.drop(columns=cols,axis=1)

    # Scale numerical Numerical features
    scaler = StandardScaler()
    scaled_arr = scaler.fit_transform(df)
    data_scaled = pd.DataFrame(scaled_arr, columns = df.columns)
    df =data_scaled

    return df

In [61]:
#one last function to do all the work
def clean_data(data,attributes):
    '''
    Clean data from beginning to end
  
    '''
    # clean description data
    print("Cleaning attributes data ...")
    new_attributes = identify_nas(attributes)
    print('==============================')
    # delete Nas
    print("Deleting Nas columns ...")
    data = drop_cols(data)
    print('==============================')
    # replace Unknown by  Nas
    print("Replacing Unknowns by Nas ...")
    data = unknown_nas(data,new_attributes)
    print('==============================')
    # input nas
    print("Imputing data ...")
    data = impute_nas(data,new_attributes)
    print('==============================')
    # encode data
    print("Encoding data ...")
    data = encode_data(data,new_attributes)
    
    data.fillna(method='ffill',inplace=True)
    print('==============================')
    print('Final data size: ',data.shape)
    
    return data

In [59]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 356 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(356)
memory usage: 13.6 MB


In [62]:
encode_data(customers,attributes)

TypeError: ignored

In [55]:
#lets now clean and encode  our data
population = clean_data(population, attributes)
customers = clean_data(customers,attributes)

Cleaning attributes data ...
Deleting Nas columns ...
Old Data shape:  (5000, 356)
New Data shape:  (5000, 356)
Replacing Unknowns by Nas ...
Imputing data ...
Number of Nas before filling : 0
Number of Nas after filling Numeric features : 0
Number of Nas after  filling All features: 0
Encoding data ...


TypeError: ignored

In [36]:
customers.select_dtypes(include='object').nunique()

CAMEO_DEU_2015               45
CAMEO_DEUG_2015              19
CAMEO_INTL_2015              43
D19_LETZTER_KAUF_BRANCHE     34
EINGEFUEGT_AM               319
OST_WEST_KZ                   2
PRODUCT_GROUP                 3
CUSTOMER_GROUP                2
dtype: int64

In [37]:
population.select_dtypes(include='object').nunique()

CAMEO_DEU_2015               44
CAMEO_DEUG_2015              18
CAMEO_INTL_2015              42
D19_LETZTER_KAUF_BRANCHE     34
EINGEFUEGT_AM               212
OST_WEST_KZ                   2
dtype: int64

## Exploratory Data Analysis

Our data is clean, lets check out some variable characteristics

In [ ]:
#


3. Encode Features.
        a. One hot encoding
        b. Transformational Binary encoding
4. Scale data, to standardize the variability in the features

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

In [33]:
#obtaining the best features
pca = PCA() # init pca
pca.fit(population) # fit the dataset into pca model

num_components = len(pca.explained_variance_ratio_)
ind = np.arange(num_components)
cumulativeValue = pca.explained_variance_ratio_.cumsum()

#we can plot the cumulative value as below
plt.figure(figsize= (10, 8)) # size of the chart(size of the vectors)
plt.plot(ind, cumulativeValue, marker = 'o', linestyle="--");

ValueError: ignored

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')